In [1]:
import json
import requests
import datetime

In [2]:
def load_config(filename, key):
    with open(filename, 'r') as f:
        return json.load(f)[key]
        
def update_refresh_token(refresh_file, config_file):
    token = load_config(refresh_file, 'refresh_token')
    client_id = load_config(config_file, 'client_id')
    endpoint = load_config(config_file, 'endpoint')

    parameters = {
        'grant_type': 'refresh_token',
        'refresh_token': token,
        'access_type': 'offline',
        'code': '',
        'client_id': client_id
    }
    response = requests.post(url=endpoint, data=parameters).json()
    response['expiration_time'] = (datetime.datetime.now()+datetime.timedelta(seconds=response['refresh_token_expires_in'])).strftime('%Y%m%d_%H:%M:%S')
    with open('./Data/refresh_token.json', 'w') as f:
        json.dump(response, f)

def update_access_token(refresh_file, config_file):
    token = load_config(refresh_file, 'refresh_token')
    client_id = load_config(config_file, 'client_id')
    endpoint = load_config(config_file, 'endpoint')
    
    parameters = {
        'grant_type': r'refresh_token',
        'refresh_token': token,
        'access_type': '',
        'code': '',
        'client_id': client_id
    }
    response = requests.post(url=endpoint, data=parameters).json()
    response['expiration_time'] = (datetime.datetime.now()+datetime.timedelta(seconds=response['expires_in'])).strftime('%Y%m%d_%H:%M:%S')
    with open('./Data/access_token.json', 'w') as f:
        json.dump(response, f)

def update_token(refresh_file, access_file, config_file):
    refresh_expiration = load_config(refresh_file, 'expiration_time')
    access_expiration = load_config(access_file, 'expiration_time')
    refresh_expires = datetime.datetime.strptime(refresh_expiration, '%Y%m%d_%H:%M:%S')-datetime.datetime.now()
    access_expires = datetime.datetime.strptime(access_expiration, '%Y%m%d_%H:%M:%S')-datetime.datetime.now()
    
    if refresh_expires < datetime.timedelta(seconds=604800):
        update_refresh_token(refresh_file, config_file)
        print('Refresh token updated.')
    if access_expires < datetime.timedelta(seconds=360):   
        update_access_token(refresh_file, config_file)
        print('Access token updated.')
    else:
        print("Tokens are still active. Refresh expires in {}. Access expires in: {}".format(refresh_expires, access_expires))

In [3]:
config_file = './Data/config.json'
refresh_file = './Data/refresh_token.json'
access_file = './Data/access_token.json'

In [4]:
update_token(refresh_file, access_file, config_file)

Tokens are still active. Refresh expires in 89 days, 23:58:22.270975. Access expires in: 0:28:22.270950
